In [1]:
# import torch

import csv
import sys
import time
import os
from collections import Counter
from tqdm import tqdm
import pandas as pd

from transformers import BertTokenizer, BertModel, BertForMaskedLM
# import torch.nn.functional as F

In [2]:
objects = ['bread', 'fish', 'flowers', 'glasses', 'meat', 'paintings']
jobs = ['baker','fisherman','florist','optician','butcher','painter']

job2object = {j:o for j,o in zip(jobs,objects)}

entities = ["Daniel", "Sebastian", "Joe", "Rowan", "Jack", "John", "Jake"]

verb = "works as a"

q_template = "For his job , entity sells [MASK] ."

In [ ]:
objects = ['bread', 'fish', 'flowers', 'glasses', 'meat', 'paintings']
jobs = ['father','grandfather','uncle','newphew','older brother','']

job2object = {j:o for j,o in zip(jobs,objects)}

entities = ["Daniel", "Sebastian", "Joe", "Rowan", "Jack", "John", "Jake"]

verb = "works as a"

q_template = "For his job , entity sells [MASK] ."

## num_attractors = 0

In [3]:
def no_attractors(entities, verb, q_template, relation_dict):
    answers = []
    sentences = []
    sub_sents = single_entity_subsent(entities, verb, relation_dict)
    for sentence in sub_sents:
        entity, verb, job, object = sentence
        sentence = " ".join([entity, verb, job])
        question = q_template.replace("entity", entity)
        answers.append(object)
        sentences.append(" ".join([sentence, question]))

    return sentences, answers

def single_entity_subsent(entities, verb, relation_dict):
    sub_sents = []
    for entity in entities:
        for job, object in relation_dict.items():
            sentence = (entity, verb, job, object)
            sub_sents.append(sentence)

    return sub_sents

In [4]:
s,a = no_attractors(entities, verb, q_template, job2object)

## num_attractors >= 1

In [ ]:
num_attractors = 1

In [ ]:
answers = []
sentences = []
sub_sents = single_entity_subsent(entities, verb, job2object)
for sentence in sub_sents:
    entity, verb, job, object = sentence
    sentence_0 = " ".join([entity, verb, job])
    question = q_template.replace("entity", entity)
    sents = []
    sents.append(sentence_0)
    for i in range(num_attractors):



        if i == num_attractors - 1:
            "and"



    sents.append(question)
    answers.append(object)
    sentences.append(" ".join(sents))

## simple 3-word SVO(adj) with verb fixed

In [1]:
import numpy as np
import itertools
import utils
choice = np.random.choice

In [2]:
def SVO(subject_pool, verb, object_pool, num_attractors):
    assert(num_attractors+1 <= len(subject_pool))
    assert(num_attractors+1 <= len(object_pool))
    subjects = choice(subject_pool, num_attractors+1, replace=False)
    objects = choice(object_pool, num_attractors+1, replace=False)
    sentences = []
    answer = objects[0]

    context = [subjects[0], verb, objects[0]]
    if num_attractors == 0: context.append(".")
    sentences.extend(context)
    for i in range(0, num_attractors):
        sent = [",", subjects[i+1], verb, objects[i+1]]
        if i == num_attractors - 1:
            sent.append(".")
            sent[0] = "and"
        sentences.extend(sent)

    question = ["How", "does", subjects[0], verb, "[MASK]", "."]
    sentences.extend(question)
    return " ".join(sentences), answer

In [65]:
def SVO(subject_pool, verb, object_pool, num_attractors):
    assert(num_attractors+1 <= len(subject_pool))
    assert(num_attractors+1 <= len(object_pool))
    sentences = []
    answers = []

    combs = list(itertools.permutations(object_pool, r=num_attractors+1))
    for comb in combs:
        if len(np.unique(comb))==1: continue
        sentence = []
        context = [subject_pool[0], verb, comb[0]]
        if num_attractors == 0: context.append(".")
        sentence.extend(context)
        for i in range(num_attractors):
            sent = [",", subject_pool[i+1], verb, comb[i+1]]
            if i == num_attractors - 1:
                sent.append(".")
                sent[0] = "and"
            sentence.extend(sent)
        question = ["How", "does", subject_pool[0], verb[:-1], "[MASK]", "."]
        sentence.extend(question)
        sentence = " ".join(sentence)

        sentences.append(sentence)
        answers.append(comb[0])
    return sentences, answers

In [75]:
subject_pool = ["Mary", "Daniel", "Joe", "Sarah", "Jack", "Alex", "Sam"]
verb = "feels"
object_pool = ["happy", "sad", "angry", "bored", "scared", "confused", "anxious"]

In [76]:
s,a = SVO(subject_pool, verb, object_pool, 1)

In [77]:
utils.to_csv(s,a,"raw_data/simple_SVO/emotions/no_repetition/num_attractors=1.csv")

In [117]:
his_name = ["Caspian","Wynn","Harry","Sebastian"]
her_name = ["Zenobia","Michelle","Oriana","Annabeth"]
my_name = ["Sky","Taylor","Saffron","Keith"]
your_name = ["Mitchell", "Ted", "Monica", "Veronica"]

In [111]:
def name_1(names, subject_pool):
    verb = "is"
    sentences = []
    answers = []

    # combs = list(itertools.combinations_with_replacement(np.arange(num_attractors+1), r=2))
    combs = [(i,j)for i in range(4) for j in range(4)]
    for comb in combs:
        sentence = []
        context = [subject_pool[0], verb, names[0][comb[0]]]
        sentence.extend(context)
        sent = ["and", subject_pool[1], verb, names[1][comb[1]], "."]
        sentence.extend(sent)
        question = [subject_pool[0], verb, "[MASK]", "."]
        sentence.extend(question)
        sentence = " ".join(sentence)

        sentences.append(sentence)
        answers.append(names[0][comb[0]])
    return sentences, answers

In [125]:
subject_pool = ["his name", "her name", "my name", "your name"]
s,a = name_1([his_name,her_name], subject_pool)

In [126]:
subject_pool = ["her name", "his name", "my name", "your name"]
s1,a1 = name_1([her_name,his_name], subject_pool)

In [127]:
s = s+s1
a = a +a1
utils.to_csv(s,a,"raw_data/simple_SVO/names/num_attractors=1.csv")

In [132]:
len(female_names)

129000

# Explore data set

In [56]:
data_dir = 'data/combined_data'
subdirs = [f for f in os.listdir(data_dir)]
models = ['BertBase',
 'BertLarge',
 'GPT2Large',
 'GPT2Medium',
 'GPT2Small',
 'GPT2XL',
 'RobertaBase',
 'RobertaLarge']
model = models[0]
csv_prefix = 'complete_data_For_MultipleEntityObjectDistractorAccuracy'
csv_name = csv_prefix + model + '.csv'

In [77]:
stats = {}
sentences = {''}
targets = {''}
for subdir in tqdm(subdirs):
    stats[subdir] = {'num_items':0, 'text_length': 0, 'num_attractors':0, 
    'by_num_attractors':{
        0:{'num_items':0, 'text_length': 0, 'distance_from_beginning':0, 'distance_from_masked_token':0}, 
        1:{'num_items':0, 'text_length': 0, 'distance_from_beginning':0, 'distance_from_masked_token':0}, 
        2:{'num_items':0, 'text_length': 0, 'distance_from_beginning':0, 'distance_from_masked_token':0}, 
        3:{'num_items':0, 'text_length': 0, 'distance_from_beginning':0, 'distance_from_masked_token':0}
        }
    }
    
    # n_a_set = {0}
    f_path = os.path.join(data_dir, subdir, model, csv_name)
    reader = csv.DictReader(open(f_path), delimiter='\t')
    for row in reader:
        sentence = row['sentence']
        sentences.add(sentence)
        targets.add(row['target_occupation'])
        text_length = len(sentence.split())
        num_attractors = int(row['count_attractors'])
        # n_a_set.add(num_attractors)
        distance_from_beginning = int(row['distance_from_beginning'])
        distance_from_masked_token = int(row['distance_from_masked_token'])

        stats[subdir]['num_items'] += 1
        stats[subdir]['text_length'] += text_length
        stats[subdir]['num_attractors'] += num_attractors

        stats[subdir]['by_num_attractors'][num_attractors]['num_items'] += 1
        stats[subdir]['by_num_attractors'][num_attractors]['text_length'] += text_length
        stats[subdir]['by_num_attractors'][num_attractors]['distance_from_beginning'] += distance_from_beginning
        stats[subdir]['by_num_attractors'][num_attractors]['distance_from_masked_token'] += distance_from_masked_token
        
    n = stats[subdir]['num_items']  
    stats[subdir]['num_attractors'] /= n
    stats[subdir]['text_length'] /= n
    for num_attractors in [0,1,2,3]:
        n = stats[subdir]['by_num_attractors'][num_attractors]['num_items']
        stats[subdir]['by_num_attractors'][num_attractors]['text_length'] /= n
        stats[subdir]['by_num_attractors'][num_attractors]['distance_from_beginning'] /= n
        stats[subdir]['by_num_attractors'][num_attractors]['distance_from_masked_token'] /= n
        


100%|██████████| 11/11 [00:07<00:00,  1.47it/s]


In [76]:
len(sentences)

82691

In [78]:
targets

{'',
 'beijing',
 'bread',
 'century',
 'egypt',
 'fish',
 'flowers',
 'france',
 'glasses',
 'goal',
 'helsinki',
 'india',
 'italy',
 'jakarta',
 'meat',
 'paintings',
 'paris',
 'peru',
 'run',
 'russia',
 'santiago',
 'touchdown',
 'warsaw'}

In [73]:
stats

{'multiple_entity_distractor': {'num_items': 12896,
  'text_length': 34.61879652605459,
  'num_attractors': 2.594292803970223,
  'by_num_attractors': {0: {'num_items': 176,
    'text_length': 20.386363636363637,
    'distance_from_beginning': 8.886363636363637,
    'distance_from_masked_token': 7.5},
   1: {'num_items': 816,
    'text_length': 25.470588235294116,
    'distance_from_beginning': 8.75,
    'distance_from_masked_token': 12.720588235294118},
   2: {'num_items': 3072,
    'text_length': 31.440104166666668,
    'distance_from_beginning': 8.75,
    'distance_from_masked_token': 18.690104166666668},
   3: {'num_items': 8832,
    'text_length': 36.85326086956522,
    'distance_from_beginning': 8.75,
    'distance_from_masked_token': 24.10326086956522}}},
 'multiple_entity_distractorSwapped': {'num_items': 12896,
  'text_length': 38.04978287841191,
  'num_attractors': 2.594292803970223,
  'by_num_attractors': {0: {'num_items': 176,
    'text_length': 19.613636363636363,
    'dist

In [63]:
df = {'csv_file': [], 'num_items':[], 'text_length': []}
for csv_file, data in stats.items():
    df['csv_file'].append(csv_file)
    df['num_items'].append(data['num_items'])
    df['text_length'].append(data['text_length'])

In [71]:
df = {'csv_file': [], 'num_items':[], 'text_length': [], 'num_items_0':[], 'text_length_0': [], 'num_items_1':[], 'text_length_1': [], 'num_items_2':[], 'text_length_2': [], 'num_items_3':[], 'text_length_3': []}
for csv_file, data in stats.items():
    df['csv_file'].append(csv_file)
    df['num_items'].append(data['num_items'])
    df['text_length'].append(data['text_length'])
    df['num_items_0'].append(data['by_num_attractors'][0]['num_items'])
    df['text_length_0'].append(data['by_num_attractors'][0]['text_length'])
    df['num_items_1'].append(data['by_num_attractors'][1]['num_items'])
    df['text_length_1'].append(data['by_num_attractors'][1]['text_length'])
    df['num_items_2'].append(data['by_num_attractors'][2]['num_items'])
    df['text_length_2'].append(data['by_num_attractors'][2]['text_length'])
    df['num_items_3'].append(data['by_num_attractors'][3]['num_items'])
    df['text_length_3'].append(data['by_num_attractors'][3]['text_length'])

In [72]:
df = pd.DataFrame.from_dict(df)
df

,csv_file,num_items,text_length,num_items_0,text_length_0,num_items_1,text_length_1,num_items_2,text_length_2,num_items_3,text_length_3
0,multiple_entity_distractor,12896,34.618797,176,20.386364,816,25.470588,3072,31.440104,8832,36.853261
1,multiple_entity_distractorSwapped,12896,38.049783,176,19.613636,816,28.860294,3072,34.338542,8832,40.557065
2,multiple_entity_object,6448,33.743176,88,20.386364,408,25.210784,1536,30.786458,4416,35.826087
3,multiple_entity_profession,6448,35.494417,88,20.386364,408,25.730392,1536,32.093750,4416,37.880435
4,multiple_entity_with_neutral_distractor,7568,35.803911,88,20.386364,440,26.150000,1760,32.550000,5280,37.950000
5,single_entity_distractor,12896,30.131824,176,20.386364,816,25.735294,3072,28.190104,8832,31.407609
6,single_entity_distractorSwapped,12896,29.772333,176,19.613636,816,25.720588,3072,27.791667,8832,31.038043
7,single_entity_object,6448,28.856079,88,20.386364,408,25.328431,1536,27.302083,4416,29.891304
8,single_entity_profession,6448,31.407568,88,20.386364,408,26.142157,1536,29.078125,4416,32.923913
9,single_entity_with_neutral_distractor,7568,33.207452,88,20.386364,440,25.490909,1760,30.550000,5280,34.950000


In [4]:
def ordered_items_to_list(items):
    return candidate.strip('[').strip(']').replace("'",'').replace(' ','').split(',')

In [5]:
f = open(data_dir)
reader = csv.DictReader(f, delimiter='\t')

ct = 0
targets = []
sentences = []
candidates = []
num_attractors = []

for row in reader:
    sentence = row['sentence']
    target = row['target_occupation']
    candidate = row['ordered_items']
    n_attractors = row['count_attractors']
    
    targets.append(target)
    sentences.append(sentence)
    candidates.append(ordered_items_to_list(candidate))
    num_attractors.append(n_attractors)
    ct += 1
    if ct < 10:
        print("No. {} Sentence: {} Target: {}".format(ct, sentence, target))

No. 1 Sentence: Daniel works as a florist . For his job , Daniel sells [MASK] . Target: flowers
No. 2 Sentence: Daniel has a sister and now works as a florist . For his job , Daniel sells [MASK] . Target: flowers
No. 3 Sentence: Daniel has a sister , played basketball , and now works as a florist . For his job , Daniel sells [MASK] . Target: flowers
No. 4 Sentence: Daniel has a sister , played basketball , sang in a choir , and now works as a florist . For his job , Daniel sells [MASK] . Target: flowers
No. 5 Sentence: Sebastian works as an optician . For his job , Sebastian sells [MASK] . Target: glasses
No. 6 Sentence: Sebastian has a sister and now works as an optician . For his job , Sebastian sells [MASK] . Target: glasses
No. 7 Sentence: Sebastian has a sister , played basketball , and now works as an optician . For his job , Sebastian sells [MASK] . Target: glasses
No. 8 Sentence: Sebastian has a sister , played basketball , sang in a choir , and now works as an optician . For h

In [6]:
targets_counter = Counter(targets)

candidate_targets = Counter()
for candidate in candidates:
    candidate_targets.update(candidate)


print(f"Number of instances: {ct}")
print(candidate_targets)
print(targets_counter)

Number of instances: 12896
Counter({'flowers': 4128, 'paintings': 4128, 'fish': 4128, 'glasses': 4128, 'meat': 4128, 'bread': 4128, 'santiago': 4128, 'paris': 4128, 'beijing': 4128, 'warsaw': 4128, 'jakarta': 4128, 'helsinki': 4128, 'India': 4128, 'Egypt': 4128, 'France': 4128, 'Italy': 4128, 'Peru': 4128, 'Russia': 4128, 'goal': 512, 'touchdown': 512, 'run': 512, 'century': 512})
Counter({'flowers': 688, 'glasses': 688, 'meat': 688, 'bread': 688, 'fish': 688, 'paintings': 688, 'santiago': 688, 'beijing': 688, 'helsinki': 688, 'paris': 688, 'jakarta': 688, 'warsaw': 688, 'india': 688, 'france': 688, 'egypt': 688, 'peru': 688, 'italy': 688, 'russia': 688, 'touchdown': 128, 'run': 128, 'goal': 128, 'century': 128})
